In [3]:
import os
import time

import dropbox
from dropbox.exceptions import AuthError

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

from dotenv import load_dotenv

load_dotenv()

DROPBOX_NAMESPACE_ID = os.environ.get("DROPBOX_NAMESPACE_ID")
EMAIL = os.environ.get("DROPBOX_USER_MAIL")
PWD = "Start2storewithDropboxOAI10!"
APP_KEY = os.environ.get("APP_KEY")
APP_SECRET = os.environ.get("APP_SECRET")


flow = dropbox.DropboxOAuth2FlowNoRedirect(APP_KEY, APP_SECRET)
# Have the user sign in and authorize this token
authorize_url = flow.start()

def init_selenium_driver():
    caps = DesiredCapabilities.CHROME
    caps['goog:loggingPrefs'] = {'performance': 'ALL'}
    chromedriver_path = "/opt/homebrew/bin/chromedriver"

    chrome_options = Options()
    chrome_options.add_argument("--disk-cache-size=1073741824")
    chrome_options.add_argument("--media-cache-size=1073741824")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    # options.add_argument("--headless")
    # driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver_path)
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

    return driver

driver = init_selenium_driver()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/alecioc/.wdm/drivers/chromedriver/mac64_m1/99.0.4844.51/chromedriver] found in cache
/var/folders/dt/fd9qffvj52ldl0l0_06wkkz40000gn/T/ipykernel_22181/1990434464.py:48: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [30]:
driver.get(authorize_url)

In [31]:
frame = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//iframe[@id='consent-iframe']"))
)
driver.switch_to.frame(frame)

In [33]:
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//button[@id='_evidon-decline-button']"))
).click()

In [35]:
driver.switch_to.default_content()

In [36]:
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((
        By.XPATH,
        '//input[@type="email"]'
    ))
).send_keys(EMAIL)

In [37]:
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((
        By.XPATH,
        '//input[@type="password"]'
    ))
).send_keys(PWD)

In [38]:
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((
        By.XPATH,
        '//button[@type="submit"]'
    ))
).click()

In [46]:
driver.get(authorize_url)

In [47]:
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((
        By.XPATH,
        '//button[@type="reset"]'
    ))
).click()

In [48]:
WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((
        By.XPATH,
        '//button[@name="allow_access"]'
    ))
).click()


In [49]:
auth_code = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((
        By.XPATH,
        '//input[@type="text"]'
    ))
).get_attribute("data-token")

'xOZRS9JcJG4AAAAAAAAF-bv5bMon7NZLG4eWfPU40fY'

In [ ]:
auth_code

In [50]:
# This will fail if the user enters an invalid authorization code
oauth_flow_dbx_obj = flow.finish(auth_code)
print(oauth_flow_dbx_obj)

OAuth2FlowNoRedirectResult(sl.BFMoQtfSlMKWHyYr-J2X9n1Ehb6o8auGpg95U9T3PyQma41_ThXy-pHQAsNeTdLJtOJt3-UIc1NiXZNY1J2GX4nTDKtGXwmpb0S3P0QTI4y5sHWMLewITxg2k_1cx6O3m2mUbJbitXpD, dbtid:AACULwIrT5GBnZZD7Iu7eN6LSBSGOzyQezY, , , 2022-04-06 21:00:12.326444, account_info.read account_info.write contacts.read contacts.write events.read file_requests.read file_requests.write files.content.read files.content.write files.metadata.read files.metadata.write files.permanent_delete files.team_metadata.write groups.read groups.write members.delete members.read members.write sessions.list sessions.modify sharing.read sharing.write team_data.member team_data.team_space team_info.read)


In [51]:
oauth_flow_dbx_obj.user_id

''

In [52]:
oauth_flow_dbx_obj.access_token

'sl.BFMoQtfSlMKWHyYr-J2X9n1Ehb6o8auGpg95U9T3PyQma41_ThXy-pHQAsNeTdLJtOJt3-UIc1NiXZNY1J2GX4nTDKtGXwmpb0S3P0QTI4y5sHWMLewITxg2k_1cx6O3m2mUbJbitXpD'

In [53]:
oauth_flow_dbx_obj.scope

'account_info.read account_info.write contacts.read contacts.write events.read file_requests.read file_requests.write files.content.read files.content.write files.metadata.read files.metadata.write files.permanent_delete files.team_metadata.write groups.read groups.write members.delete members.read members.write sessions.list sessions.modify sharing.read sharing.write team_data.member team_data.team_space team_info.read'

In [61]:
dbx_team = dropbox.dropbox_client.DropboxTeam(
    oauth_flow_dbx_obj.access_token
)
#print(dbx_team.team_get_info())

team_member_info = dbx_team.team_members_get_info(
    [dropbox.team.UserSelectorArg("email", EMAIL)]
)[0].get_member_info()

#print(team_member_info.profile)
print(team_member_info.profile.team_member_id)

dbmid:AAAKI-2bsP_Bh6713-Dz931ksKMka0OBw_A


In [108]:
for email in [
    #"alessandro@ciociola.me",
    #"marco.torlaschi@occupyai.tech",
    #"lucabaggi.uni@gmail.com",
    #"info@marziacamarda.it",
    "simone.rossi1199@gmail.com"
]:

    team_member_info = dbx_team.team_members_get_info(
        [dropbox.team.UserSelectorArg("email", email)]
    )[0].get_member_info()

    #print(team_member_info.profile)
    print(team_member_info.profile.email, team_member_info.profile.team_member_id)

simone.rossi1199@gmail.com dbmid:AACU5JUEskY0plddsoAbStxGkyYlHxwrDDM


In [109]:
team_member_id = team_member_info.profile.team_member_id
dbx = dropbox.dropbox_client.DropboxTeam(
    oauth_flow_dbx_obj.access_token
).with_path_root(dropbox.common.PathRoot.namespace_id(DROPBOX_NAMESPACE_ID)).as_user(
    team_member_id=team_member_id
)

In [110]:
print(dbx.users_get_current_account())

FullAccount(account_id='dbid:AAAaj7WK1gUbPSEYibRUY_rDTp9lTUCe5PI', account_type=AccountType('business', None), country='IT', disabled=False, email='simone.rossi1199@gmail.com', email_verified=True, is_paired=False, locale='en', name=Name(abbreviated_name='SR', display_name='Simone Rossi', familiar_name='Simone', given_name='Simone', surname='Rossi'), profile_photo_url=NOT_SET, referral_link='https://www.dropbox.com/referrals/AACyyvFyHEIg_wwmVT6bGEu18xPZyIUpz48?src=app9-3008337', root_info=TeamRootInfo(home_namespace_id='10168773152', home_path='/Simone Rossi', root_namespace_id='9401968128'), team=FullTeam(id='dbtid:AACULwIrT5GBnZZD7Iu7eN6LSBSGOzyQezY', name='Occupy AI', office_addin_policy=OfficeAddInPolicy('enabled', None), sharing_policies=TeamSharingPolicies(shared_folder_join_policy=SharedFolderJoinPolicy('from_anyone', None), shared_folder_member_policy=SharedFolderMemberPolicy('anyone', None), shared_link_create_policy=SharedLinkCreatePolicy('default_public', None))), team_membe

In [123]:
list_folder_results = dbx.files_list_folder("/k2")

In [124]:
list_folder_results

ListFolderResult(cursor='AAE18Kwx4uwDP9wsxlkPnxTxu-7FhCY55HywzzxAGm-A5b7xikUrGK97h6FOcaBTGnor5t4DqqMJ9JDvmOIKe4I7IO654EDQvtCnZJDYA5BOca2oS-KJPBggXFOPt79Thq4cjlgLrlre5BfPDrH3kIQr2LvanDrc6LBhoczRH51ce7QzFTTQelSzDo8kLl3anwQk10Imzk0txOI_diEfotTgfzBaD4y_p0c9zknXkzPmSiHGVqg3gS-Rtb8jauKXm6beZImzfF9qWSDj22Hk6xlL3pJBxgacP2SanBsxj5n7bg', entries=[FolderMetadata(id='id:SB_A3-rGVJwAAAAAAADdzQ', name='metadata', parent_shared_folder_id='10169967056', path_display='/k2/metadata', path_lower='/k2/metadata', property_groups=NOT_SET, shared_folder_id=NOT_SET, sharing_info=FolderSharingInfo(no_access=False, parent_shared_folder_id='10169967056', read_only=False, shared_folder_id=NOT_SET, traverse_only=False)), FolderMetadata(id='id:SB_A3-rGVJwAAAAAAADdzg', name='raw_photos', parent_shared_folder_id='10169967056', path_display='/k2/raw_photos', path_lower='/k2/raw_photos', property_groups=NOT_SET, shared_folder_id=NOT_SET, sharing_info=FolderSharingInfo(no_access=False, parent_shared_folder_id='101699670

In [125]:
#while list_folder_results.has_more:

files = list_folder_results.entries
print(files)
for file in files:
    print(file.path_display)

#client = dropbox.Dropbox(access_token)
#print('successfully linked account: ', client.users_get_current_account().name.display_name)
#with open(DROPBOX_TOKEN_PATH, 'w') as fout:
#    fout.write(access_token)

[FolderMetadata(id='id:SB_A3-rGVJwAAAAAAADdzQ', name='metadata', parent_shared_folder_id='10169967056', path_display='/k2/metadata', path_lower='/k2/metadata', property_groups=NOT_SET, shared_folder_id=NOT_SET, sharing_info=FolderSharingInfo(no_access=False, parent_shared_folder_id='10169967056', read_only=False, shared_folder_id=NOT_SET, traverse_only=False)), FolderMetadata(id='id:SB_A3-rGVJwAAAAAAADdzg', name='raw_photos', parent_shared_folder_id='10169967056', path_display='/k2/raw_photos', path_lower='/k2/raw_photos', property_groups=NOT_SET, shared_folder_id=NOT_SET, sharing_info=FolderSharingInfo(no_access=False, parent_shared_folder_id='10169967056', read_only=False, shared_folder_id=NOT_SET, traverse_only=False)), FolderMetadata(id='id:Er23tj3YP_gAAAAAAAAAAQ', name='processed_photos', parent_shared_folder_id='10169967056', path_display='/k2/processed_photos', path_lower='/k2/processed_photos', property_groups=NOT_SET, shared_folder_id='10213325872', sharing_info=FolderSharingI